Download database: 
[CatApp](https://cmr.fysik.dtu.dk/_downloads/716b1e0826acbb3d80675c116a2cb8a6/catapp.db)

In [6]:
from ase.db import connect
import numpy as np
import pandas as pd

# Connect to the CatApp database
catapp=connect('catapp.db')

def reaction(catapp,reverse=False,**kwargs):
    """Download reactions from CatApp. ab -> a + b if reverse=False
        Parameters :
            catapp : database
                The CatApp database
            reverse : bool
                if reverse=False: ab -> a + b or reverse=True: a + b -> ab
            kwargs : dict
                A dictionary with a, b, ab and the surface
        Returns : 
            output : list
                Output is a list with the reaction, the surface, 
                energy of the reaction and the activation energy
    """
    output=[]
    for row in catapp.select(**kwargs):
        er=row.er
        ea=row.ea
        ea=np.max([0,er,ea])
        if reverse==True:
            er=-er
            ea+=er
            # The activation energy must be the largest
            ea=np.max([0,er,ea])
            react='{} + {} -> {}'.format(row.a,row.b,row.ab)
            surf=row.surface
        else:
            react='{} -> {} + {}'.format(row.ab,row.a,row.b)
            surf=row.surface
        output.append((react,surf,er,ea))
    return output
        
def reaction_dict(catapp,reverse=False,react_list=[],pd_react=None,**kwargs):
    """A wrapper that makes a dict of the data from CatApp
        Parameters :
            catapp : database
                The CatApp database
            reverse : bool
                if reverse=False: ab -> a + b or reverse=True: a + b -> ab
            react_list : list of str
                A list of all the reactions (can be continued in a new function call)
            pd_react : dict
                A dictionary with the surfaces, energy differences and activation energies
                that can be continued in a new function call
            kwargs : dict
                A dictionary with a, b, ab and the surface
        Returns : 
            pd_react : dict
                A dictionary with the surface, 
                energy of the reaction and the activation energy
            react_list : list
                A list of all the reactions
    """
    if pd_react is None:
        pd_react={'Surface':[],'E_diff':[],'Ea':[]}
    for r in reaction(catapp,reverse=reverse,**kwargs):
        react,surf,er,ea=list(r)
        pd_react['Surface'].append(surf)
        if react not in react_list:
            react_list.append(react)
        pd_react['E_diff'].append(er)
        pd_react['Ea'].append(ea)
    return pd_react,react_list
    
        
# Surfaces investigated
surfaces=['Pt(111)','Cu(111)']
pd_list=[]
react_list=[]

# The CO+*->CO* elementary reaction
pd_dict={'Surface':[],'E_diff':[],'Ea':[]} 
for surf in surfaces:
    pd_dict,react_list=reaction_dict(catapp,reverse=True,react_list=react_list,\
                                     pd_react=pd_dict,**{'ab':'CO*','a':'CO', 'b':'*','surface':surf})

pd_list.append(pd.DataFrame(pd_dict).set_index('Surface'))

# The O2+2*->2O* elementary reaction
pd_dict={'Surface':[],'E_diff':[],'Ea':[]} 
for surf in surfaces:
    pd_dict,react_list=reaction_dict(catapp,reverse=False,react_list=react_list,\
                                     pd_react=pd_dict,**{'ab':'O2','surface':surf})

pd_list.append(pd.DataFrame(pd_dict).set_index('Surface'))

# Merge the pandas dataframes
pd_list=pd.concat(pd_list,axis=1,keys=react_list)
display(pd_list)


CO + * -> CO*      O2 -> O* + O*      
               E_diff   Ea        E_diff    Ea
Surface                                       
Pt(111)         -1.24  0.0         -1.49  0.62
Cu(111)         -0.36  0.0         -2.59  0.01